In [34]:
from dse.cluster import Cluster, GraphExecutionProfile, EXEC_PROFILE_GRAPH_DEFAULT
from dse.graph import GraphOptions, GraphProtocol
from dse_graph import DseGraph

from gremlin_python.process.graph_traversal import __
from gremlin_python.process.traversal import P

from math import radians, cos, sin, asin, sqrt


In [80]:
graph_name = "trees"
ep = GraphExecutionProfile(graph_options=GraphOptions(graph_name=graph_name,
                                                      graph_protocol=GraphProtocol.GRAPHSON_3_0))
cluster = Cluster(execution_profiles={EXEC_PROFILE_GRAPH_DEFAULT: ep})
session = cluster.connect()
g = DseGraph.traversal_source(session=session)

In [91]:
def create_weighted_send_edges(nearest_neighbors, g, label, key):
    # get the top 10 nearest sensors
    for sensor, neighbors in nearest_neighbors.items(): # we are always making edges from a sensor to something else
        # neighbors is a list of tuples
        for neighbor in neighbors:
            # neighbor[0] is the distance
            # neighbor[1] is the dictionary of the object
            neighbor_v = neighbor[1]
            # add an Weighted_Send edge with initialized weight of 0.10
            v1 = g.V().has("Sensor", "sensorName", sensor).next()
            v2 = g.V().has(label, key, neighbor_v.get("id")).next()
            g.V(v1). \
              addE('Weighted_Send'). \
              to(v2). \
              property("probability", "0.10"). \
              next() 
    
    
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r


def top_neighbors(sensors, compare_list, num_neighbors):
    nearest_neighbors = {}
    for sensor in sensors:
        sensor_name = sensor.get("id")
        nearest_neighbors[sensor_name] = {}
        for entity in compare_list:
            if entity.get("id") != sensor_name: # do not consider adding edges to itself
                dist = haversine(sensor.get("long"),
                                          sensor.get("lat"),
                                          entity.get("long"),
                                          entity.get("lat"))
                nearest_neighbors[sensor_name][dist] = entity
        nearest_neighbors[sensor_name] = list(sorted(nearest_neighbors[sensor_name].items()))
        nearest_neighbors[sensor_name] = nearest_neighbors[sensor_name][0:num_neighbors]
    return nearest_neighbors

        
def init_eligibility_weights(g):
    ## Get all Sensors
    sensors = g.V().hasLabel("Sensor"). \
                    project("id", "lat", "long"). \
                      by("sensorName"). \
                      by("latitude"). \
                      by("longitude"). \
                      toList()
    towers = g.V().hasLabel("Tower"). \
                   project("id", "lat", "long"). \
                      by("towerName"). \
                      by("latitude"). \
                      by("longitude"). \
                      toList()
    sensor_sensor_neighbors = top_neighbors(sensors, sensors, 10)
    sensor_tower_neighbors = top_neighbors(sensors, towers, 10)

    # initialize the sensor -- Weighted_Send --> sensor edges
    create_weighted_send_edges(sensor_sensor_neighbors, g, "Sensor", "sensorName")
    # initialize the sensor -- Weighted_Send --> tower edges
    create_weighted_send_edges(sensor_tower_neighbors, g, "Tower", "towerName")


In [92]:
init_eligibility_weights(g)

In [96]:
data = g.V().has("Sensor", "sensorName", "1308319").outE("Weighted_Send").valueMap().iterate()

In [97]:
data

[['V'], ['has', 'Sensor', 'sensorName', '1308319'], ['outE', 'Weighted_Send'], ['valueMap'], ['none'], ['values', '_ipython_canary_method_should_not_exist_'], ['values', '_ipython_canary_method_should_not_exist_']]

In [99]:
g.V().has("Sensor", "sensorName", "1308319").values("lat")

[['V'], ['has', 'Sensor', 'sensorName', '1308319'], ['values', 'lat'], ['values', '_ipython_canary_method_should_not_exist_'], ['values', '_ipython_canary_method_should_not_exist_']]